In [5]:
import numpy as np
import pandas as pd

df_raw = pd.read_csv('train.csv')
df_raw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [14]:
import numpy as np
import pandas as pd


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from math import sqrt
from scipy import stats
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder


import matplotlib.pyplot as plt
import matplotlib as matplotlib
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')

def data_process(df_raw,remove_outlier = False,remove_hard_to_fit = False,linear_model = False, get_dummies=False, label_encode=False ):

	# Make a copy so the original dataframe will not be altered.
    df_processed = df_raw.copy()
    
    
	# Remove outliers.
    outlier_list = [524, 1299, 463, 31, 534, 1433, 739, 1159, 108, 1231, 971, 1424]
    df_processed = df_processed.drop(outlier_list)

    
    ## Missing values
    
    # 259 LotFrontage  - replace missing value with 0 
    df_processed.LotFrontage = df_processed.LotFrontage.fillna(0)

    # 1369 Alley - replace with None
    df_processed.Alley = df_processed.Alley.fillna('None')

    # 8 MasVnrType and MasVnrArea - replace MasVnrType with None and MasVnrArea with 0
    df_processed.MasVnrType = df_processed.MasVnrType.fillna('None')
    df_processed.MasVnrArea = df_processed.MasVnrArea.fillna(0)

    # 37 basement: BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2- replace with Nb
    df_processed.BsmtQual = df_processed.BsmtQual.fillna('Nb')
    df_processed.BsmtCond = df_processed.BsmtCond.fillna('Nb')
    df_processed.BsmtExposure = df_processed.BsmtExposure.fillna('Nb')
    df_processed.BsmtFinType1 = df_processed.BsmtFinType1.fillna('Nb')
    df_processed.BsmtFinType2 = df_processed.BsmtFinType2.fillna('Nb')
    df_processed.TotalBsmtSF = df_processed.TotalBsmtSF.fillna(0)
    

    # 690 FireplaceQu - replace with Nf
    df_processed.FireplaceQu = df_processed.FireplaceQu.fillna('Nf')

    # 81 Garage: GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond - replace with Ng and year with 0 
    df_processed.GarageType = df_processed.GarageType.fillna('Ng')
    df_processed.GarageFinish = df_processed.GarageFinish.fillna('Ng')
    df_processed.GarageQual = df_processed.GarageQual.fillna('Ng')
    df_processed.GarageCond = df_processed.GarageCond.fillna('Ng')
    df_processed.GarageYrBlt = df_processed.GarageYrBlt.fillna(0)

    # 1453 PoolQC - replace with Np
    df_processed.PoolQC = df_processed.PoolQC.fillna('Np')

    # 1179 Fence - replace with Nf
    df_processed.Fence = df_processed.Fence.fillna('Nf')

    # 1406 MiscFeature - replace with None    
    df_processed.MiscFeature = df_processed.MiscFeature.fillna('None')

    # 1 Electrical
    df_processed = df_processed[pd.notnull(df_processed.Electrical)]

    ## Combine columns and drop multicollinear columns 
    
    # combine bathroom quanlitity 
    df_processed['BsmtBath'] = df_processed.BsmtFullBath + df_processed.BsmtHalfBath * 0.5
    df_processed['Bath'] = df_processed.FullBath + df_processed.HalfBath * 0.5
    df_processed = df_processed.drop(['BsmtFullBath', 'BsmtHalfBath','FullBath','HalfBath'], axis=1)

    # drop TotalBsmtSF - multicollinearaty
    #df_processed = df_processed.drop(['TotalBsmtSF'], axis=1)

    # drop GrLivArea - multicollinearaty
    #df_processed = df_processed.drop(['GrLivArea'], axis=1)

    # drop GarageArea - higher correlation than GarageACars, results are better as well
    df_processed = df_processed.drop(['GarageArea'], axis=1) 
    
    
	# Feature Transformation - take the logarithm of the features.
    #Linear_Num_Cols = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'LotArea', 'GarageArea', 'TotRmsAbvGrd', 'TotalSF', 'BsmtFinSF1']
    df_processed.SalePrice = np.log(df_processed.SalePrice)
    df_processed.GrLivArea = np.log(df_processed.GrLivArea)
    df_processed.TotalBsmtSF = np.log(df_processed.TotalBsmtSF+1)
#     df_processed.LotArea = np.log(df_processed.LotArea) -- performance decreases
#     df_processed.GarageArea = np.log(df_processed.GarageArea)



	# Categorical Features Processsing

	# MSSubClass processing - MSSubClass 20-90 contains only duplicate information with HouseStyle and YearBuilt.
    df_processed['MSSubClass'] = df_processed['MSSubClass'].replace(['20','30','40','45','50','60','70','75','80','85'], '0')

    # Convert numerical to categorical. 
    df_processed[['MSSubClass','OverallQual','OverallCond']] = df_processed[['MSSubClass','OverallQual','OverallCond']].astype(str)

    #Get Dummies 
    
    if get_dummies:
        df_processed = pd.get_dummies(df_processed, columns=df_processed.select_dtypes(include=['object']).columns, drop_first=True)
    
    
    #get label encoder. categorical data change to numerical values
    if label_encode:
        le = LabelEncoder()
        categorical_ordinal_col=df_processed.select_dtypes(include=['object']).columns.to_list()
        df_processed[categorical_ordinal_col]=df_processed[categorical_ordinal_col].apply(lambda col: le.fit_transform(col))

   #---Multiply features: 
#     df_processed["add_OverallGrade"] = df_processed["OverallQual"] * df_processed["OverallCond"]
#     df_processed["add_GarageGrade"] = df_processed["GarageQual"] * df_processed["GarageCond"]
#     df_processed["add_ExterGrade"] = df_processed["ExterQual"] * df_processed["ExterCond"]
#     df_processed["add_KitchenScore"] = df_processed["KitchenAbvGr"] * df_processed["KitchenQual"]
#     df_processed["add_FireplaceScore"] = df_processed["Fireplaces"] * df_processed["FireplaceQu"]
#     df_processed["add_GarageScore"] = df_processed["GarageArea"] * df_processed["GarageQual"]
#     df_processed["add_PoolScore"] = df_processed["PoolArea"] * df_processed["PoolQC"]
#     df_processed['add_GrLivArea*OvQual'] = df_processed['GrLivArea'] * df_processed['OverallQual']
#     df_processed['add_QualOverall*Exter*Kitch*Bsmt*Garage'] = df_processed['OverallQual'] * df_processed['ExterQual'] * df_processed['KitchenQual'] * df_processed['BsmtQual'] * df_processed['GarageQual']



    return df_processed



In [15]:
# categorical_ordinal_col=['Alley',
#  'BldgType',
#  'BsmtCond',
#  'BsmtExposure',
#  'BsmtFinType1',
#  'BsmtFinType2',
#  'BsmtQual',
#  'CentralAir',
#  'Condition1',
#  'Condition2',
#  'Electrical',
#  'ExterCond',
#  'ExterQual',
#  'Exterior1st',
#  'Exterior2nd',
#  'Fence',
#  'FireplaceQu',
#  'Foundation',
#  'Functional',
#  'GarageCond',
#  'GarageFinish',
#  'GarageQual',
#  'GarageType',
#  'Heating',
#  'HeatingQC',
#  'HouseStyle',
#  'KitchenQual',
#  'LandContour',
#  'LandSlope',
#  'LotConfig',
#  'LotShape',
#  'MSSubClass',
#  'MSZoning',
#  'MasVnrType',
#  'MiscFeature',
#  'Neighborhood',
#  'OverallCond',
#  'OverallQual',
#  'PavedDrive',
#  'PoolQC',
#  'RoofMatl',
#  'RoofStyle',
#  'SaleCondition',
#  'SaleType',
#  'Street',
#  'Utilities']

In [39]:
# #List of numerical columns
# numerical_col=['LotFrontage',
#  'LotArea',
#  'YearBuilt',
#  'YearRemodAdd',
#  'MasVnrArea',
#  'BsmtFinSF1',
#  'BsmtFinSF2',
#  'BsmtUnfSF',
#  '1stFlrSF',
#  '2ndFlrSF',
#  'LowQualFinSF',
#  'BedroomAbvGr',
#  'KitchenAbvGr',
#  'TotRmsAbvGrd',
#  'Fireplaces',
#  'GarageYrBlt',
#  'GarageCars',
#  'WoodDeckSF',
#  'OpenPorchSF',
#  'EnclosedPorch',
#  '3SsnPorch',
#  'ScreenPorch',
#  'PoolArea',
#  'MiscVal',
#  'MoSold',
#  'YrSold',
#  'SalePrice',
#  'BsmtBath',
#  'Bath']

In [38]:
#data prep
data_processed_label_encode=data_process(df_raw,label_encode=True)
data_processed_label_encode.head()

TypeError: data_process() got an unexpected keyword argument 'label_encode'

In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)
    

def model_random_forest_sklearn(data_processed, 
                                n_estimators,
                                max_features,
                                max_depth):
    
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data_processed)

    # The predicted column is "SalePrice" .
    train_x = train.drop(["SalePrice"], axis=1)
    test_x = test.drop(["SalePrice"], axis=1)
    train_y = train[["SalePrice"]]
    test_y = test[["SalePrice"]]
 
 #The number of jobs to run in parallel for both fit and predict.
    rfr = RandomForestRegressor(n_jobs=1, random_state=0, oob_score=True)
    param_grid = {'n_estimators': n_estimators, # number of trees
                  'max_features': max_features, #The number of features to consider when looking for the best split
                  'max_depth':max_depth} #max depth of the tree
    
    model = GridSearchCV(estimator=rfr, param_grid=param_grid, n_jobs=1,cv=5)
    model.fit(train_x, train_y.values.ravel())
    
    print('Random forecast regression...')
    print('Best Params:')
    print(model.best_params_)
    
    #Mean cross-validated score of the best_estimator
    print('Best CV Score:') 
    print(model.best_score_)
    
    y_pred = model.predict(test_x)
    print ('test model r2:',model.score(test_x,test_y))
    print ('train model r2:', model.score(train_x, train_y))
    
    #eval_metrics(actual, predicted)
    (rmse, mae, r2) = eval_metrics(test_y, y_pred)
    
   # print ('oob score:', model.oob_score_ )
    
    # Print out metrics
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)
    

    # Log parameter, metrics, and model to MLflow
    mlflow.log_param("max_depth", model.best_params_.get('max_depth'))
    mlflow.log_param("max_features", model.best_params_.get('max_features'))
    mlflow.log_param("n_estimators", model.best_params_.get('n_estimators'))
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    #mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(rfr, "model")


In [21]:
#Run random forest

# def model_random_forest_sklearn(data_processed, 
#                                 n_estimators,
#                                 max_features,
#                                 max_depth):
model_random_forest_sklearn(data_processed_label_encode, [100],[5,8,10], [20,25,10,5] )

Random forecast regression...
Best Params:
{'max_depth': 20, 'max_features': 10, 'n_estimators': 100}
Best CV Score:
0.829757167561785
test model r2: 0.8137184741407288
train model r2: 0.9756807061139423


AttributeError: 'GridSearchCV' object has no attribute 'oob_score_'

In [32]:
#Run random forest

# def model_random_forest_sklearn(data_processed, 
#                                 n_estimators,
#                                 max_features,
#                                 max_depth):
model_random_forest_sklearn(data_processed_label_encode,[100],[5,8,10], [20,25,10,5] )

Random forecast regression...
Best Params:
{'max_depth': 25, 'max_features': 10, 'n_estimators': 100}
Best CV Score:
0.8582240300622415
test model r2: 0.8548783370637293
train model r2: 0.9802158088439499
  RMSE: 0.14927884894154003
  MAE: 0.10084586789784643
  R2: 0.8548783370637294


In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
import warnings
import sys
from sklearn.model_selection import RandomizedSearchCV

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


rf = RandomForestRegressor()
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]


  
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data_processed_label_encode)

# The predicted column is "SalePrice" .
train_x = train.drop(["SalePrice"], axis=1)
test_x = test.drop(["SalePrice"], axis=1)
train_y = train[["SalePrice"]]
test_y = test[["SalePrice"]]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
              }
print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(train_x, train_y)

rf_random.best_params_

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  9.2min finished
C:\Users\Laptop2\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [33]:
from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [None],
    'max_features': [6,7,8,9],
    'min_samples_leaf': [1,2,3,4],
    'min_samples_split': [2,3,4],
    'n_estimators': list(range(390,450))
}# Create a based model
rf = RandomForestRegressor()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(train_x, train_y)
grid_search.best_params_




Fitting 3 folds for each of 2880 candidates, totalling 8640 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 18.1min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 21.2min
[Parallel(n_jobs=-1)]: Done 4893 tasks      | elapsed: 25.8min
[Parallel(n_jobs=-1)]: Done 5824 tasks      | elapsed: 30.2min
[Parallel(n_jobs=-1)]: Done 6837 tasks      | elapsed: 35.4min
[Parallel(n_jobs=-1)]: Done 7930 tasks      | elapsed: 41.2min
[Parallel(n_jobs=-1)]: Done 8640 out of 8640 |

{'bootstrap': False,
 'max_depth': None,
 'max_features': 9,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 396}

In [ ]:
#Mean cross-validated score of the best_estimator
print('Best CV Score:') 
print(grid_search.best_score_)

y_pred = grid_search.predict(test_x)
print ('test model r2:',grid_search.score(test_x,test_y))
print ('train model r2:', grid_search.score(train_x, train_y))
    
#eval_metrics(actual, predicted)
(rmse, mae, r2) = eval_metrics(test_y, y_pred)

print ('oob score:', grid_search.oob_score_ )

# Print out metrics
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)